## NOTES:
* this didn't work very well, but is the best version of this algorithm yet
* we probably want to rethink this module entirely

In [ ]:
import json
import os
import pandas as pd
from collections import Counter

## full test 1000 CSAS paragraphs per language

In [39]:
df_tokens = pd.read_csv("token_testing/token_retry_debug.csv")
token_error_counts = Counter()

for i, row in df_tokens.iterrows():
    for t in row['missing_tokens'].split():
        token = "".join([x for x in t if x.isalpha()])
        token_error_counts[token] +=1

print(token_error_counts)

Counter({'NOMENCLATURE': 5530, 'ACRONYM': 1773, 'TAXON': 1097})


In [91]:
total_tokens = Counter()

def collect_untranslated_files(folder_path):
    contents_list = []
    
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt') and '_translated' not in filename:
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                if "_en" in filename:
                    contents_list.append(("en", file.read()))
                if "_fr" in filename:
                    contents_list.append(("fr", file.read()))
    
    return contents_list

with open("../../Data/preferential_translations.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

nomenclature_fr = data['translations']['nomenclature']
nomenclature_en = {v: k for k, v in nomenclature_fr.items()}

taxon_fr = data['translations']['taxon']
taxon_en = {v: k for k, v in taxon_fr.items()}

acronym_fr = data['translations']['acronym']
acronym_en = {v: k for k, v in acronym_fr.items()}

site_fr = data['translations']['site']
site_en = {v: k for k, v in site_fr.items()}

untranslated_text = collect_untranslated_files("token_testing/paragraphs/")
len(untranslated_text)

2021

In [92]:
lookups = {
    "en": {"NOMENCLATURE": nomenclature_en, "TAXON": taxon_en, "ACRONYM": acronym_en, "SITE": site_en},
    "fr": {"NOMENCLATURE": nomenclature_fr, "TAXON": taxon_fr, "ACRONYM": acronym_fr, "SITE": site_fr}
}

for lang, text in untranslated_text:
    for token_type, lookup_dict in lookups[lang].items():
        total_tokens[token_type] += sum(text.count(key) for key in lookup_dict)

total_tokens

Counter({'NOMENCLATURE': 7318, 'ACRONYM': 3951, 'TAXON': 274, 'SITE': 4})

In [ ]:
# translation error percentages 
total_attempts = 8  # number of repeats that could make it into the error output

(
    format(token_error_counts["NOMENCLATURE"] / total_tokens["NOMENCLATURE"] / total_attempts, "0.1%"),
    format(token_error_counts["TAXON"] / total_tokens["TAXON"] / total_attempts, "0.1%"),
    format(token_error_counts["ACRONYM"] / total_tokens["ACRONYM"] / total_attempts, "0.1%"),
    format(token_error_counts["SITE"] / total_tokens["SITE"] / total_attempts, "0.1%"),
)

# this is not working very well

('9.4%', '50.0%', '5.6%', '0.0%')

# find and replace errors (for paragraph level translations)

In [ ]:
# don't recalculate this stuff if we run all cells
raise ValueError("You. Shall. Not. Pass!!!")

In [2]:
# hand picked examples
errors = [
    {
        'NOMENCLATURE0001': {'original_text': 'débarquements', 'category': 'nomenclature', 'translation': 'landings', 'should_translate': True},
        'NOMENCLATURE0002': {'original_text': 'fréquence', 'category': 'nomenclature', 'translation': 'frequency', 'should_translate': True},
        'TAXON0001': {'original_text': 'homard', 'category': 'taxon', 'translation': 'Lobster', 'should_translate': True},
        'pre_processed_text': 'Les indicateurs dépendants de la pêche présentés comprennent les NOMENCLATURE0001 de la pêche dirigée et des prises accessoires, les prises standardisées par effort unitaire (UEPC) et le nombre estimatif de jours de pêche requis pour débarquer 9 000 kg. Les indicateurs dépendants de la pêche comprennent un relevé des pièges CPUE, un programme de surveillance de la longueur et de la NOMENCLATURE0002 du quai et un programme de bio-collecteur pour évaluer le recrutement. De plus, les résultats d’un modèle d’abondance de prédateurs de TAXON0001 sont discutés.',
        'translated_text_with_error': 'The fishery-dependent indicators presented include directed fisheries and bycatch NOMENCLATURE0001, unit effort standardized catches (UEPCs) and the estimated number of fishing days required to land 9,000 kg. The fishery-dependent indicators include a survey of CPUE traps, a dock length and NOMENCLATURE0002 monitoring program, and a model bio-collector program to assess the TAXs.',
    },
    {
        'NOMENCLATURE0001': {'original_text': 'débarquements', 'category': 'nomenclature', 'translation': 'landings', 'should_translate': True},
        'NOMENCLATURE0002': {'original_text': 'fréquence', 'category': 'nomenclature', 'translation': 'frequency', 'should_translate': True},
        'TAXON0001': {'original_text': 'homard', 'category': 'taxon', 'translation': 'Lobster', 'should_translate': True},
        'pre_processed_text': 'Les indicateurs dépendants de la pêche présentés comprennent les NOMENCLATURE0001 de la pêche dirigée et des prises accessoires, les prises standardisées par effort unitaire (UEPC) et le nombre estimatif de jours de pêche requis pour débarquer 9 000 kg. Les indicateurs dépendants de la pêche comprennent un relevé des pièges CPUE, un programme de surveillance de la longueur et de la NOMENCLATURE0002 du quai et un programme de bio-collecteur pour évaluer le recrutement. De plus, les résultats d’un modèle d’abondance de prédateurs de TAXON0001 sont discutés.',
        'translated_text_with_error': 'The fishery-dependent indicators presented include directed and bycatch NOMENCLATURE0001, unit effort standardized catches (UEPCs) and the estimated number of days of fishing required to land 9,000 kg. The fishery-dependent indicators include a survey of CPUE traps, a dock length and NOMENCLATURE0002 monitoring program and a model bio-collector program to assess the TAXs.',
    },
    {
        'NOMENCLATURE0001': {'original_text': 'débarquements', 'category': 'nomenclature', 'translation': 'landings', 'should_translate': True},
        'NOMENCLATURE0002': {'original_text': 'fréquence', 'category': 'nomenclature', 'translation': 'frequency', 'should_translate': True},
        'TAXON0001': {'original_text': 'homard', 'category': 'taxon', 'translation': 'Lobster', 'should_translate': True},
        'pre_processed_text': 'Les indicateurs dépendants de la pêche présentés comprennent les NOMENCLATURE0001 de la pêche dirigée et des prises accessoires, les prises standardisées par effort unitaire (UEPC) et le nombre estimatif de jours de pêche requis pour débarquer 9 000 kg. Les indicateurs dépendants de la pêche comprennent un relevé des pièges CPUE, un programme de surveillance de la longueur et de la NOMENCLATURE0002 du quai et un programme de bio-collecteur pour évaluer le recrutement. De plus, les résultats d’un modèle d’abondance de prédateurs de TAXON0001 sont discutés.',
        'translated_text_with_error': 'The fishery-dependent indicators presented include directed fisheries and bycatch NOMENCLATURE0001, unit effort standardized catches (UEPCs) and the estimated number of fishing days required to land 9,000 kg. The fishery-dependent indicators include a survey of CPUE traps, a wharf length and NOMENCLATURE0002 monitoring program, and a model bio-collector program to assess the TAXs.',
    },
    {
        'NOMENCLATURE0001': {'original_text': 'débarquements', 'category': 'nomenclature', 'translation': 'landings', 'should_translate': True},
        'NOMENCLATURE0002': {'original_text': 'fréquence', 'category': 'nomenclature', 'translation': 'frequency', 'should_translate': True},
        'TAXON0001': {'original_text': 'homard', 'category': 'taxon', 'translation': 'Lobster', 'should_translate': True},
        'pre_processed_text': 'Les indicateurs dépendants de la pêche présentés comprennent les NOMENCLATURE0001 de la pêche dirigée et des prises accessoires, les prises standardisées par effort unitaire (UEPC) et le nombre estimatif de jours de pêche requis pour débarquer 9 000 kg. Les indicateurs dépendants de la pêche comprennent un relevé des pièges CPUE, un programme de surveillance de la longueur et de la NOMENCLATURE0002 du quai et un programme de bio-collecteur pour évaluer le recrutement. De plus, les résultats d’un modèle d’abondance de prédateurs de TAXON0001 sont discutés.',
        'translated_text_with_error': 'The fishery-dependent indicators presented include directed fisheries and bycatch NOMENCLATURE0001, unit effort standardized catches (UEPCs) and the estimated number of fishing days required to land 9,000 kg. The fishery-dependent indicators include a survey of CPUE traps, a dock length and NOMENCLATURE0002 monitoring program, and a model bio-collector program to assess the TAXs.',
    },
    {
        'NOMENCLATURE0001': {'original_text': 'débarquements', 'category': 'nomenclature', 'translation': 'landings', 'should_translate': True},
        'NOMENCLATURE0002': {'original_text': 'fréquence', 'category': 'nomenclature', 'translation': 'frequency', 'should_translate': True},
        'TAXON0001': {'original_text': 'homard', 'category': 'taxon', 'translation': 'Lobster', 'should_translate': True},
        'pre_processed_text': 'Les indicateurs dépendants de la pêche présentés comprennent les NOMENCLATURE0001 de la pêche dirigée et des prises accessoires, les prises standardisées par effort unitaire (UEPC) et le nombre estimatif de jours de pêche requis pour débarquer 9 000 kg. Les indicateurs dépendants de la pêche comprennent un relevé des pièges CPUE, un programme de surveillance de la longueur et de la NOMENCLATURE0002 du quai et un programme de bio-collecteur pour évaluer le recrutement. De plus, les résultats d’un modèle d’abondance de prédateurs de TAXON0001 sont discutés.',
        'translated_text_with_error': 'The fishery-dependent indicators presented include directed fisheries and bycatch NOMENCLATURE0001, unit effort standardized catches (UEPCs) and the estimated number of fishing days required to land 9,000 kg. The fishery-dependent indicators include a survey of CPUE traps, a dock length and NOMENCLATURE0002 monitoring program, and a model bio-collector program to assess the TAXs.',
    },
    {
        'NOMENCLATURE0001': {'original_text': 'débarquements', 'category': 'nomenclature', 'translation': 'landings', 'should_translate': True},
        'NOMENCLATURE0002': {'original_text': 'fréquence', 'category': 'nomenclature', 'translation': 'frequency', 'should_translate': True},
        'TAXON0001': {'original_text': 'homard', 'category': 'taxon', 'translation': 'Lobster', 'should_translate': True},
        'pre_processed_text': 'Les indicateurs dépendants de la pêche présentés comprennent les NOMENCLATURE0001 de la pêche dirigée et des prises accessoires, les prises standardisées par effort unitaire (UEPC) et le nombre estimatif de jours de pêche requis pour débarquer 9 000 kg. Les indicateurs dépendants de la pêche comprennent un relevé des pièges CPUE, un programme de surveillance de la longueur et de la NOMENCLATURE0002 du quai et un programme de bio-collecteur pour évaluer le recrutement. De plus, les résultats d’un modèle d’abondance de prédateurs de TAXON0001 sont discutés.',
        'translated_text_with_error': 'The fishery-dependent indicators presented include directed fisheries and bycatch NOMENCLATURE0001, unit effort standardized catches (UEPCs) and the estimated number of fishing days required to land 9,000 kg. The fishery-dependent indicators include a survey of CPUE traps, a dock length and NOMENCLATURE0002 monitoring program, and a model bio-collector program to assess the TAXs.',
    },
    {
        'NOMENCLATURE0001': {'original_text': 'débarquements', 'category': 'nomenclature', 'translation': 'landings', 'should_translate': True},
        'NOMENCLATURE0002': {'original_text': 'fréquence', 'category': 'nomenclature', 'translation': 'frequency', 'should_translate': True},
        'TAXON0001': {'original_text': 'homard', 'category': 'taxon', 'translation': 'Lobster', 'should_translate': True},
        'pre_processed_text': 'Les indicateurs dépendants de la pêche présentés comprennent les NOMENCLATURE0001 de la pêche dirigée et des prises accessoires, les prises standardisées par effort unitaire (UEPC) et le nombre estimatif de jours de pêche requis pour débarquer 9 000 kg. Les indicateurs dépendants de la pêche comprennent un relevé des pièges CPUE, un programme de surveillance de la longueur et de la NOMENCLATURE0002 du quai et un programme de bio-collecteur pour évaluer le recrutement. De plus, les résultats d’un modèle d’abondance de prédateurs de TAXON0001 sont discutés.',
        'translated_text_with_error': 'The fishery-dependent indicators presented include directed fisheries and bycatch NOMENCLATURE0001, unit effort standardized catches (UEPCs) and the estimated number of fishing days required to land 9,000 kg. The fishery-dependent indicators include a survey of CPUE traps, a dock length and NOMENCLATURE0002 monitoring program, and a model bio-collector program to assess the TAXs.',
    },
    {
        'NOMENCLATURE0001': {'original_text': 'débarquements', 'category': 'nomenclature', 'translation': 'landings', 'should_translate': True},
        'NOMENCLATURE0002': {'original_text': 'fréquence', 'category': 'nomenclature', 'translation': 'frequency', 'should_translate': True},
        'TAXON0001': {'original_text': 'homard', 'category': 'taxon', 'translation': 'Lobster', 'should_translate': True},
        'pre_processed_text': 'Les indicateurs dépendants de la pêche présentés comprennent les NOMENCLATURE0001 de la pêche dirigée et des prises accessoires, les prises standardisées par effort unitaire (UEPC) et le nombre estimatif de jours de pêche requis pour débarquer 9 000 kg. Les indicateurs dépendants de la pêche comprennent un relevé des pièges CPUE, un programme de surveillance de la longueur et de la NOMENCLATURE0002 du quai et un programme de bio-collecteur pour évaluer le recrutement. De plus, les résultats d’un modèle d’abondance de prédateurs de TAXON0001 sont discutés.',
        'translated_text_with_error': 'The fishery-dependent indicators presented include directed fisheries and bycatch NOMENCLATURE0001, unit effort standardized catches (UEPCs) and the estimated number of fishing days required to land 9,000 kg. The fishery-dependent indicators include a survey of CPUE traps, a dock length and NOMENCLATURE0002 monitoring program, and a model bio-collector program to assess the TAXs.',
    },
    {
        'NOMENCLATURE0001': {'original_text': 'débarquements', 'category': 'nomenclature', 'translation': 'landings', 'should_translate': True},
        'NOMENCLATURE0002': {'original_text': 'fréquence', 'category': 'nomenclature', 'translation': 'frequency', 'should_translate': True},
        'TAXON0001': {'original_text': 'homard', 'category': 'taxon', 'translation': 'Lobster', 'should_translate': True},
        'pre_processed_text': 'Les indicateurs dépendants de la pêche présentés comprennent les NOMENCLATURE0001 de la pêche dirigée et des prises accessoires, les prises standardisées par effort unitaire (UEPC) et le nombre estimatif de jours de pêche requis pour débarquer 9 000 kg. Les indicateurs dépendants de la pêche comprennent un relevé des pièges CPUE, un programme de surveillance de la longueur et de la NOMENCLATURE0002 du quai et un programme de bio-collecteur pour évaluer le recrutement. De plus, les résultats d’un modèle d’abondance de prédateurs de TAXON0001 sont discutés.',
        'translated_text_with_error': 'The fishery-dependent indicators presented include directed fisheries and bycatch NOMENCLATURE0001, unit effort standardized catches (UEPCs) and the estimated number of fishing days required to land 9,000 kg. The fishery-dependent indicators include a survey of CPUE traps, a dock length and NOMENCLATURE0002 monitoring program, and a model bio-collector program to assess the TAXs.',
    },
    {
        'NOMENCLATURE0001': {'original_text': 'débarquements', 'category': 'nomenclature', 'translation': 'landings', 'should_translate': True},
        'NOMENCLATURE0002': {'original_text': 'fréquence', 'category': 'nomenclature', 'translation': 'frequency', 'should_translate': True},
        'TAXON0001': {'original_text': 'homard', 'category': 'taxon', 'translation': 'Lobster', 'should_translate': True},
        'pre_processed_text': 'Les indicateurs dépendants de la pêche présentés comprennent les NOMENCLATURE0001 de la pêche dirigée et des prises accessoires, les prises standardisées par effort unitaire (UEPC) et le nombre estimatif de jours de pêche requis pour débarquer 9 000 kg. Les indicateurs dépendants de la pêche comprennent un relevé des pièges CPUE, un programme de surveillance de la longueur et de la NOMENCLATURE0002 du quai et un programme de bio-collecteur pour évaluer le recrutement. De plus, les résultats d’un modèle d’abondance de prédateurs de TAXON0001 sont discutés.',
        'translated_text_with_error': 'The fishing-dependent indicators presented include the NOMENCLATURE0001 of direct fishing and accessory fishing, the standardized unit effort fishing (UEPC) and the estimated number of fishing days required to land 9,000 kg. The fishing-dependent indicators include a CPUE fishing report, a length monitoring program and the NOMENCLATURE0002 of the cave and a bio-collector program to evaluate recruitment.',
    },
    {
        'NOMENCLATURE0001': {'original_text': 'débarquements', 'category': 'nomenclature', 'translation': 'landings', 'should_translate': True},
        'NOMENCLATURE0002': {'original_text': 'fréquence', 'category': 'nomenclature', 'translation': 'frequency', 'should_translate': True},
        'TAXON0001': {'original_text': 'homard', 'category': 'taxon', 'translation': 'Lobster', 'should_translate': True},
        'pre_processed_text': 'Les indicateurs dépendants de la pêche présentés comprennent les NOMENCLATURE0001 de la pêche dirigée et des prises accessoires, les prises standardisées par effort unitaire (UEPC) et le nombre estimatif de jours de pêche requis pour débarquer 9 000 kg. Les indicateurs dépendants de la pêche comprennent un relevé des pièges CPUE, un programme de surveillance de la longueur et de la NOMENCLATURE0002 du quai et un programme de bio-collecteur pour évaluer le recrutement. De plus, les résultats d’un modèle d’abondance de prédateurs de TAXON0001 sont discutés.',
        'translated_text_with_error': 'The presented fishing-dependent indicators include the NOMENCLATURE0001 of direct fishing and accessory fishing, standardized unit effort fishing (UEPC) and the estimated number of fishing days required to land 9 000 kg. The fishing-dependent indicators include a CPUE fishing report, a long and long monitoring program and a NOMENCLATURE0002 of the cave and a bio-collector program to evaluate recruitment.',
    },
    {
        'NOMENCLATURE0001': {'original_text': 'débarquements', 'category': 'nomenclature', 'translation': 'landings', 'should_translate': True},
        'NOMENCLATURE0002': {'original_text': 'fréquence', 'category': 'nomenclature', 'translation': 'frequency', 'should_translate': True},
        'TAXON0001': {'original_text': 'homard', 'category': 'taxon', 'translation': 'Lobster', 'should_translate': True},
        'pre_processed_text': 'Les indicateurs dépendants de la pêche présentés comprennent les NOMENCLATURE0001 de la pêche dirigée et des prises accessoires, les prises standardisées par effort unitaire (UEPC) et le nombre estimatif de jours de pêche requis pour débarquer 9 000 kg. Les indicateurs dépendants de la pêche comprennent un relevé des pièges CPUE, un programme de surveillance de la longueur et de la NOMENCLATURE0002 du quai et un programme de bio-collecteur pour évaluer le recrutement. De plus, les résultats d’un modèle d’abondance de prédateurs de TAXON0001 sont discutés.',
        'translated_text_with_error': 'The fishing-dependent indicators presented include the NOMENCLATURE0001 of direct fishing and accessory fishing, the standardized unit effort fishing (UEPC) and the estimated number of fishing days required to land 9,000 kg. The fishing-dependent indicators include a CPUE fishing report, a length monitoring program and the NOMENCLATURE0002 of the cave and a bio-collector program to evaluate recruitment.',
    },
    {
        'NOMENCLATURE0001': {'original_text': 'débarquements', 'category': 'nomenclature', 'translation': 'landings', 'should_translate': True},
        'NOMENCLATURE0002': {'original_text': 'fréquence', 'category': 'nomenclature', 'translation': 'frequency', 'should_translate': True},
        'TAXON0001': {'original_text': 'homard', 'category': 'taxon', 'translation': 'Lobster', 'should_translate': True},
        'pre_processed_text': 'Les indicateurs dépendants de la pêche présentés comprennent les NOMENCLATURE0001 de la pêche dirigée et des prises accessoires, les prises standardisées par effort unitaire (UEPC) et le nombre estimatif de jours de pêche requis pour débarquer 9 000 kg. Les indicateurs dépendants de la pêche comprennent un relevé des pièges CPUE, un programme de surveillance de la longueur et de la NOMENCLATURE0002 du quai et un programme de bio-collecteur pour évaluer le recrutement. De plus, les résultats d’un modèle d’abondance de prédateurs de TAXON0001 sont discutés.',
        'translated_text_with_error': 'The fishing-dependent indicators presented include the NOMENCLATURE0001 of direct fishing and accessory fishing, the standardized unit effort fishing (UEPC) and the estimated number of fishing days required to land 9,000 kg. The fishing-dependent indicators include a CPUE fishing report, a length monitoring program and the NOMENCLATURE0002 of the cave and a bio-collector program to evaluate recruitment.',
    },
    {
        'NOMENCLATURE0001': {'original_text': 'débarquements', 'category': 'nomenclature', 'translation': 'landings', 'should_translate': True},
        'NOMENCLATURE0002': {'original_text': 'fréquence', 'category': 'nomenclature', 'translation': 'frequency', 'should_translate': True},
        'TAXON0001': {'original_text': 'homard', 'category': 'taxon', 'translation': 'Lobster', 'should_translate': True},
        'pre_processed_text': 'Les indicateurs dépendants de la pêche présentés comprennent les NOMENCLATURE0001 de la pêche dirigée et des prises accessoires, les prises standardisées par effort unitaire (UEPC) et le nombre estimatif de jours de pêche requis pour débarquer 9 000 kg. Les indicateurs dépendants de la pêche comprennent un relevé des pièges CPUE, un programme de surveillance de la longueur et de la NOMENCLATURE0002 du quai et un programme de bio-collecteur pour évaluer le recrutement. De plus, les résultats d’un modèle d’abondance de prédateurs de TAXON0001 sont discutés.',
        'translated_text_with_error': 'The fishing-dependent indicators presented include the NOMENCLATURE0001 of direct fishing and accessory fishing, the standardized unit effort fishing (UEPC) and the estimated number of fishing days required to land 9 000 kg. The fishing-dependent indicators include a report of the CPUE traps, a program of monitoring the length and the NOMENCLATURE0002 of the cave and a bio-collector program to evaluate recruitment.',
    },
    {
        'NOMENCLATURE0001': {'original_text': 'débarquements', 'category': 'nomenclature', 'translation': 'landings', 'should_translate': True},
        'NOMENCLATURE0002': {'original_text': 'fréquence', 'category': 'nomenclature', 'translation': 'frequency', 'should_translate': True},
        'TAXON0001': {'original_text': 'homard', 'category': 'taxon', 'translation': 'Lobster', 'should_translate': True},
        'pre_processed_text': 'Les indicateurs dépendants de la pêche présentés comprennent les NOMENCLATURE0001 de la pêche dirigée et des prises accessoires, les prises standardisées par effort unitaire (UEPC) et le nombre estimatif de jours de pêche requis pour débarquer 9 000 kg. Les indicateurs dépendants de la pêche comprennent un relevé des pièges CPUE, un programme de surveillance de la longueur et de la NOMENCLATURE0002 du quai et un programme de bio-collecteur pour évaluer le recrutement. De plus, les résultats d’un modèle d’abondance de prédateurs de TAXON0001 sont discutés.',
        'translated_text_with_error': 'The fishing-dependent indicators presented include the NOMENCLATURE0001 of direct fishing and accessory fishing, the standardized unit effort fishing (UEPC) and the estimated number of fishing days required to land 9,000 kg. The fishing-dependent indicators include a report of the CPUE traps, a program of monitoring the length and the NOMENCLATURE0002 of the cave and a bio-collector program to evaluate recruitment.',
    },
    {
        'NOMENCLATURE0001': {'original_text': 'débarquements', 'category': 'nomenclature', 'translation': 'landings', 'should_translate': True},
        'NOMENCLATURE0002': {'original_text': 'fréquence', 'category': 'nomenclature', 'translation': 'frequency', 'should_translate': True},
        'TAXON0001': {'original_text': 'homard', 'category': 'taxon', 'translation': 'Lobster', 'should_translate': True},
        'pre_processed_text': 'Les indicateurs dépendants de la pêche présentés comprennent les NOMENCLATURE0001 de la pêche dirigée et des prises accessoires, les prises standardisées par effort unitaire (UEPC) et le nombre estimatif de jours de pêche requis pour débarquer 9 000 kg. Les indicateurs dépendants de la pêche comprennent un relevé des pièges CPUE, un programme de surveillance de la longueur et de la NOMENCLATURE0002 du quai et un programme de bio-collecteur pour évaluer le recrutement. De plus, les résultats d’un modèle d’abondance de prédateurs de TAXON0001 sont discutés.',
        'translated_text_with_error': 'The fishing-dependent indicators presented include the NOMENCLATURE0001 of direct fishing and accessory fishing, the standardized unit effort fishing (UEPC) and the estimated number of fishing days required to land 9,000 kg. The fishing-dependent indicators include a CPUE fishing report, a length monitoring program and the NOMENCLATURE0002 of the cave and a bio-collector program to evaluate recruitment.',
    },
    {
        'NOMENCLATURE0001': {'original_text': 'débarquements', 'category': 'nomenclature', 'translation': 'landings', 'should_translate': True},
        'NOMENCLATURE0002': {'original_text': 'fréquence', 'category': 'nomenclature', 'translation': 'frequency', 'should_translate': True},
        'TAXON0001': {'original_text': 'homard', 'category': 'taxon', 'translation': 'Lobster', 'should_translate': True},
        'pre_processed_text': 'Les indicateurs dépendants de la pêche présentés comprennent les NOMENCLATURE0001 de la pêche dirigée et des prises accessoires, les prises standardisées par effort unitaire (UEPC) et le nombre estimatif de jours de pêche requis pour débarquer 9 000 kg. Les indicateurs dépendants de la pêche comprennent un relevé des pièges CPUE, un programme de surveillance de la longueur et de la NOMENCLATURE0002 du quai et un programme de bio-collecteur pour évaluer le recrutement. De plus, les résultats d’un modèle d’abondance de prédateurs de TAXON0001 sont discutés.',
        'translated_text_with_error': 'The fishing-dependent indicators presented include the NOMENCLATURE0001 of direct fishing and accessory fishing, the standardized unit effort fishing (UEPC) and the estimated number of fishing days required to land 9,000 kg. The fishing-dependent indicators include a CPUE fishing report, a length monitoring program and the NOMENCLATURE0002 of the cave and a bio-collector program to evaluate recruitment.',
    },
    {
        'NOMENCLATURE0001': {'original_text': 'débarquements', 'category': 'nomenclature', 'translation': 'landings', 'should_translate': True},
        'NOMENCLATURE0002': {'original_text': 'fréquence', 'category': 'nomenclature', 'translation': 'frequency', 'should_translate': True},
        'TAXON0001': {'original_text': 'homard', 'category': 'taxon', 'translation': 'Lobster', 'should_translate': True},
        'pre_processed_text': 'Les indicateurs dépendants de la pêche présentés comprennent les NOMENCLATURE0001 de la pêche dirigée et des prises accessoires, les prises standardisées par effort unitaire (UEPC) et le nombre estimatif de jours de pêche requis pour débarquer 9 000 kg. Les indicateurs dépendants de la pêche comprennent un relevé des pièges CPUE, un programme de surveillance de la longueur et de la NOMENCLATURE0002 du quai et un programme de bio-collecteur pour évaluer le recrutement. De plus, les résultats d’un modèle d’abondance de prédateurs de TAXON0001 sont discutés.',
        'translated_text_with_error': 'The fishing-dependent indicators presented include the NOMENCLATURE0001 of direct fishing and accessory captures, the standardized unit effort captures (UEPC) and the estimated number of fishing days required to land 9,000 kg. The fishing-dependent indicators include a CPUE traps report, a length monitoring program and the NOMENCLATURE0002 of the cave and a bio-collector program to evaluate recruitment.',
    },
]

In [3]:
error_type_counts = Counter()
verbose = False

for error in errors:
    source_text = error['pre_processed_text']
    translated_text = error['translated_text_with_error']
    tokens = [k for k in error.keys() if k not in ['pre_processed_text', 'translated_text_with_error']]
    for t in tokens:
        if t not in translated_text:
            error_type_counts[t] += 1
            if verbose:
                print('missing', t)
                print('text:')
                print(source_text)
                print(translated_text)
                print()

print(error_type_counts)

Counter({'TAXON0001': 18})


# examples of preferential translations

In [4]:
df_training = pd.read_json("../../Data/pipeline_training_data.jsonl", lines=True)

In [5]:
df_training[df_training.source.str.contains("Pacific Ocean")].iloc[0].source

'41 The National Marine Fisheries Service (NMFS) has extrapolated some data collected on other turtles in the western Pacific Ocean to Leatherbacks..'

In [6]:
things = ['Atlantic Ocean', 'CSAS', 'espèces hôtes', 'Snow Crab']

for thing in things:
    print(thing)
    source = df_training[df_training.source.str.contains(thing)].iloc[0].source
    target = df_training[df_training.source.str.contains(thing)].iloc[0].target
    print(source)
    print(target)
    print()

Atlantic Ocean
Long-Term Time Trends Year-to-year, water temperatures on the Scotian Shelf and in the Gulf of Maine are among the most variable in the North Atlantic Ocean..
Tendances temporelles à long terme Les températures de l'eau sur le plateau néo-écossais et dans le golfe du Maine sont parmi les plus variables de l'Atlantique Nord d'année en année..

CSAS
In these cases, data would still be kept and maintained at regions, with links from the CSAS site..
Dans ces cas, les données seraient gardées et maintenues au niveau régional, avec des liens au site du SCCS..

espèces hôtes
Une fois soulevée par l'auteure, on discute de l'inclusion des espèces hôtes et de leur habitat comme faisant partie de l'habitat essentiel des espèces en péril..
The inclusion of host species and their habitat as part of Critical Habitat for at risk species was discussed, once brought up by the author..

Snow Crab
CPUE is applied here as a metric of relative biomass for Snow Crab which, in NL, provides a l